In [1]:
from google.colab import userdata

# Setting up the Agent Core (API keys):
TAVILY_API_KEY = userdata.get("Tavily-key")
OPENAI_API_KEY = userdata.get("OpenAI-key")

# Loading the Agent Modules (libraries):
!pip install -q langchain openai langchain-openai langchain-tavily langchain-community

from langchain_openai import OpenAI
from langchain_tavily import TavilySearch
from langchain.tools import Tool
from langchain.memory import ConversationBufferMemory, ChatMessageHistory
from langchain.chains import LLMMathChain, LLMChain
from langchain.prompts import PromptTemplate
from langchain.agents import initialize_agent, AgentType
from langchain.agents import AgentOutputParser
from langchain.schema import AgentAction, AgentFinish
import re

# The Agent Processor (LLM):
processor = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

# The Information Acquisition Module (Search Tool):
acquirer = TavilySearch(tavily_api_key=TAVILY_API_KEY)
search_module = Tool(
    name="tavily_search",
    func=acquirer.run,
    description="This module is designed to fetch external information based on a query. Input should be a search query.",
    handle_tool_error=True
)

# The Internal State Recorder (Memory):
state_recorder = ConversationBufferMemory(memory_key="chat_history", chat_memory=ChatMessageHistory())

# The Calculation Unit (Math Chain):
calculation_unit_prompt = PromptTemplate.from_template("Evaluate: {equation}")
calculator_unit = LLMChain(llm=processor, prompt=calculation_unit_prompt)
math_unit = Tool(
    name="Calculator",
    func=calculator_unit.run,
    description="This unit is capable of performing mathematical computations.",
    handle_tool_error=True
)

# The Available Capabilities (Tools):
capabilities = [search_module, math_unit]

# The Output Router (Custom Parser):
class AgentOutputRouter(AgentOutputParser):
    def parse(self, llm_output: str) -> AgentAction | AgentFinish:
        if "Final Answer:" in llm_output:
            return AgentFinish(
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        regex = r"Action: (.*?)\nAction Input: (.*?)$"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            return AgentAction(tool="tavily_search", tool_input=llm_output, log=llm_output)
        tool = match.group(1).strip()
        tool_input = match.group(2).strip().replace("'", "")
        return AgentAction(tool=tool, tool_input=tool_input, log=llm_output)

output_router = AgentOutputRouter()

<ipython-input-1-74bffd771d10>:34: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  state_recorder = ConversationBufferMemory(memory_key="chat_history", chat_memory=ChatMessageHistory())
<ipython-input-1-74bffd771d10>:38: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  calculator_unit = LLMChain(llm=processor, prompt=calculation_unit_prompt)


**1. Simple Reflex Agents:**
This Agent reacts directly to its current percept without maintaining any internal state.

In [2]:
def simple_reflex_agent(query):
    response = search_module.run(query)
    return f"Simple Reflex Agent's response to '{query}': {response}"

print("--- Simple Reflex Agents ---")
initial_probe = "Define the core principle of a simple reflex agent."
agent_output_sr = simple_reflex_agent(initial_probe)
print(agent_output_sr)
print("-" * 30)

--- Simple Reflex Agents ---
Simple Reflex Agent's response to 'Define the core principle of a simple reflex agent.': {'query': 'Define the core principle of a simple reflex agent.', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Convolutional Neural Network CNN: What is the Rule of a Simple Reflex ...', 'url': 'https://pupuweb.com/convolutional-neural-network-cnn-what-is-the-rule-of-a-simple-reflex-agent/', 'content': 'A simple reflex agent is the most basic type of intelligent agent in artificial intelligence (AI). It operates by responding directly to environmental stimuli based on predefined rules, without considering past percepts or future consequences. The core principle governing its behavior is the condition-action rule, which can be summarized as: If', 'score': 0.902481, 'raw_content': None}, {'title': 'Understanding Simple Reflex Agents: Reactive Decision-Making in AI', 'url': 'https://www.genaicourses.com/blog/post/simple-reflex-agents', '

**2. Model-Based Reflex Agents:** This Agent maintains a basic internal model (history) to inform its actions.

In [3]:
def model_based_agent(current_input, memory_unit):
    world_model = memory_unit.load_memory_variables({})['chat_history']
    new_info = search_module.run(current_input)
    agent_response = f"Model-Based Reflex Agent (considering its model): '{world_model}', processed '{current_input}' and found: {new_info}"
    memory_unit.save_context({"input": current_input}, {"output": agent_response})
    return agent_response

print("--- Model-Based Reflex Agents ---")
first_question_mb = "What is the advantage of an agent having an internal model?"
agent_action_mb1 = model_based_agent(first_question_mb, state_recorder)
print(agent_action_mb1)

second_question_mb = "How does an agent update its internal model?"
agent_action_mb2 = model_based_agent(second_question_mb, state_recorder)
print(agent_action_mb2)
print("-" * 30)

--- Model-Based Reflex Agents ---
Model-Based Reflex Agent (considering its model): '', processed 'What is the advantage of an agent having an internal model?' and found: {'query': 'What is the advantage of an agent having an internal model?', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'What Are Model-Based Agents? | TEDAI San Francisco', 'url': 'https://tedai-sanfrancisco.ted.com/glossary/model-based-agents/', 'content': 'Model-based agents offer several advantages, including improved decision-making, adaptability, and the ability to handle complex environments. By maintaining an internal model, these agents can make more informed decisions and adapt to changes in their environment. This makes them suitable for applications where the environment is dynamic and', 'score': 0.9092728, 'raw_content': None}, {'title': 'What Are AI Agents (Intelligent agent)? How it works, Benefits', 'url': 'https://fpt.ai/blogs/ai-agents/', 'content': 'Model-Based Refl

**3. Goal-Based Agents:** This Agent has a specific objective and plans its actions to achieve that goal.

In [4]:
goal_based_agent = initialize_agent(
    llm=processor,
    tools=[search_module],
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)
desired_goal = "Explain the process by which a goal-based agent decides on an action."
goal_result = goal_based_agent.run(desired_goal)
print(f"Goal-Based Agent's analysis: {goal_result}")
print("-" * 30)

<ipython-input-4-3090b199d9ab>:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  goal_based_agent = initialize_agent(
<ipython-input-4-3090b199d9ab>:8: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  goal_result = goal_based_agent.run(desired_goal)




> Entering new AgentExecutor chain...
 A goal-based agent is an AI agent that is designed to achieve a specific goal or set of goals. It uses a decision-making process to determine the best action to take in order to achieve its goal.
Action: tavily_search
Action Input: 'goal-based agent decision-making process'
Observation: {'query': "'goal-based agent decision-making process'", 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Goal-Based Agents: Intelligent Decision-Making Driven by Objectives', 'url': 'https://www.genaicourses.com/blog/post/goal-based-agents', 'content': 'In the realm of artificial intelligence (AI), goal-based agents have emerged as a powerful approach to decision-making and problem-solving. By having a clear understanding of what they aim to achieve, goal-based agents can make informed decisions, plan their actions, and adapt to changing circumstances. Goal-based agents are AI agents that make decisions and take actions based on p

**4. Utility-Based Agents:** This Agent selects actions based on an evaluation of their potential utility.

In [5]:
utility_based_agent = initialize_agent(
    llm=processor,
    tools=capabilities,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    output_parser=output_router
)
task_utility1 = "Explain the concept of a 'utility function' in intelligent agents."
utility_result1 = utility_based_agent.run(task_utility1)
print(f"Utility-Based Agent's explanation: {utility_result1}")

task_utility2 = "How does a utility-based agent handle conflicting goals?"
utility_result2 = utility_based_agent.run(task_utility2)
print(f"Utility-Based Agent's approach: {utility_result2}")
print("-" * 30)



> Entering new AgentExecutor chain...
 A utility function is a mathematical function that maps a set of possible states to a real number, representing the desirability or utility of that state for an intelligent agent.
Action: Calculator
Action Input: Utility function
Observation:  to evaluate the model

Utility function to evaluate the model


def evaluate(model, test_loader, criterion):
    """
    Utility function to evaluate the model

    Args:
        model (torch.nn.Module): The model to be evaluated
        test_loader (torch.utils.data.DataLoader): The data loader for the test dataset
        criterion (torch.nn.Module): The loss function used for evaluation

    Returns:
        float: The average loss over the test dataset
    """
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
    

**5. Learning Agents:** This Agent can improve its performance over time by learning from its experiences.

In [6]:
agent_learning_history = {}

def learning_agent(query):
    if query in agent_learning_history:
        return f"Learning Agent recalls: '{query}' - '{agent_learning_history[query]}'"
    else:
        learned_info = search_module.run(query)
        agent_learning_history[query] = learned_info
        return f"Learning Agent acquired knowledge about '{query}': '{learned_info}'"

print("--- Learning Agents ---")
first_query_learn_ag = "What is the role of a 'learning element' in an intelligent agent?"
learning_outcome_ag1 = learning_agent(first_query_learn_ag)
print(learning_outcome_ag1)

second_query_learn_ag = "Give an example of how an agent can learn from its experiences."
learning_outcome_ag2 = learning_agent(second_query_learn_ag)
print(learning_outcome_ag2)
print("-" * 30)

--- Learning Agents ---
Learning Agent acquired knowledge about 'What is the role of a 'learning element' in an intelligent agent?': '{'query': "What is the role of a 'learning element' in an intelligent agent?", 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'What is the Purpose of Learning Agents in AI?', 'url': 'https://www.allaboutai.com/ai-agents/learning-agents/', 'content': "2. Learning Element: The Brain of the Agent. The learning element is where the true intelligence of the agent resides. This component processes the data gathered by the sensors, identifies patterns, and updates the agent's decision-making model. The learning element refines the agent's behaviour over time.", 'score': 0.90633047, 'raw_content': None}, {'title': 'What Are Learning Agents? | TEDAI San Francisco', 'url': 'https://tedai-sanfrancisco.ted.com/glossary/learning-agents/', 'content': "The four main components of a learning agent are the learning element, critic, perfo